In [60]:
!pip  install langchain langchain-openai python-dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [61]:
from dotenv import load_dotenv
load_dotenv()

True

In [62]:
# setup lang smith


In [63]:
# setup open ai client
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [64]:
# implementing the bare-bones version

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate following to italian"),
    HumanMessage(content="Hello, how are you?")
]
messages

[SystemMessage(content='Translate following to italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [65]:
result = model.invoke(messages)
result

AIMessage(content='Ciao, come stai?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 23, 'total_tokens': 30, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-a2190c1f-8c22-433e-8143-ee54f365e133-0', usage_metadata={'input_tokens': 23, 'output_tokens': 7, 'total_tokens': 30, 'input_token_details': {}, 'output_token_details': {}})

In [66]:
# Adding output parsers
# Since the output is large and contains a lot of information that we do not want to see in the output, we can use output parsers to filter out and sse only the response.

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)


'Ciao, come stai?'

In [67]:
# Chaining operations
# since we have multiple operations performed we can create a pipeline of operations by chaining them together using '|' operator.

chain = model | parser

chain.invoke(messages)

'Ciao, come stai?'

In [68]:
# adding prompt templates
# prompt templates are used to format user input to a specific template and insert system messages in parallel to process.

from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate following to {language}"


prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
template_result = prompt_template.invoke({"language": "italian", "text": "Hello, how are you?"})
template_result

ChatPromptValue(messages=[SystemMessage(content='Translate following to italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [69]:
template_result.to_messages()

[SystemMessage(content='Translate following to italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [70]:
# Improving the chain with prompt template

chain = prompt_template | model | parser 
chain

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate following to {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])
| AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f5576099760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f557609b1a0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x7f557608f530>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7f557609cce0>, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://airnd4944693893.cognitiveservices.azure.com/openai/deployments/gpt-35-turbo/chat/c

In [71]:

chain.invoke({"language": "italian", "text": "Hello, how are you?"})

'Ciao, come stai?'